In [3]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [23]:
tweet_data = pd.read_csv('dataWithPolarity.csv')
tweet_data.head()

/Users/anukritigoyal/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,id_x,screen_name,description,created_at_x,location,is_verified,...,created_at_y,tweet_text,hashtag_entities,url_entities,favorites_count,retweet_count,quoted_status_id,in_reply_to_status_id,sentiment,polarity
0,0,0,0,0,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,...,2016-07-26 15:15:46,Congrats to Chief Master Sgt Amy Giaquinto of ...,{},{https://t.co/vUJ7muCTy5},7.0,4.0,NaN,NaN,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000
1,1,1,1,0,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,...,2017-04-06 16:15:01,ICYMI I'm working on legislative initiatives t...,{},{https://medium.com/@RepStefanik/supporting-mi...,5.0,2.0,NaN,NaN,"Sentiment(polarity=-0.1, subjectivity=0.1)",-0.100000
2,2,2,2,0,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,...,2016-07-27 17:35:17,ICYMI: #NY21 Students register now to particip...,{NY21},{https://t.co/3CbW2bp1RV},1.0,2.0,NaN,NaN,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000
3,3,3,3,0,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,...,2015-11-16 17:57:40,The @GOPpolicy hearing on The Essential Role o...,"{Millennials,GOPFuture}",{https://t.co/dWFSDnFJ1e},4.0,0.0,NaN,NaN,"Sentiment(polarity=0.0, subjectivity=0.3)",0.000000
4,4,4,4,0,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,...,2017-01-03 15:59:01,Today Congress opens the 115th session. Lookin...,{NY21},{https://t.co/zT5Kj1tmCM},7.0,3.0,NaN,NaN,"Sentiment(polarity=-0.2916666666666667, subjec...",-0.291667


In [ ]:
account_creation = tweet_data['created_at_x'].tolist()
tweet_creation = tweet_data['created_at_y'].tolist()
time = account_creation[0]
# type(time)

In [ ]:
# Converting strings of date and times to Datetime objects

accounts = pd.to_datetime(tweet_data['created_at_x'], errors='coerce')
tweet_data['created_at_x'] = accounts
tweets = pd.to_datetime(tweet_data['created_at_y'], errors='coerce')
tweet_data['created_at_y'] = accounts

In [ ]:
# Attaching party of account holders

In [9]:
pol_accounts = pd.read_csv('pol_accounts.csv', sep=';')
pol_tweets = pd.read_csv('pol_tweets.csv', sep=';')

1661553

In [17]:
pol_accounts = pol_accounts[pol_accounts['array_agg'].str.contains(r"\b(?:senator|governor|congressperson)\b", regex=True)]

,id,screen_name,description,created_at,location,is_verified,latest_following_count,latest_followers_count,latest_status_count,array_agg
0,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}"
1,1058256326,RepChrisCollins,Proud to represent New York’s 27th Congression...,2013-01-03 16:55:01,NaN,t,587,10874,2916,"{congressperson,Republican,republican}"
2,17494010,SenSchumer,Official account of Senator Chuck Schumer - Ne...,2008-11-19 20:10:20,"New York, NY",t,26578,522353,10076,"{senator,Democrat,democrat}"
3,2955485182,SenatorRounds,U.S. Senator for South Dakota,2015-01-01 20:23:49,"Pierre, South Dakota",t,542,11180,1622,"{senator,Republican,republican}"
4,339852137,SupJaniceHahn,Proudly representing the 4th District of Los A...,2011-07-21 19:22:22,"San Pedro, CA",t,1738,13050,4218,"{congressperson,Democrat}"
6,248699486,RepJeffDenham,Working for the people of California's 10th di...,2011-02-07 15:01:35,"Modesto, CA",t,1642,15305,2415,"{congressperson,Republican,republican}"
7,16583468,RepMikeHonda,Former Congressman for 17th District of Califo...,2008-10-03 20:18:31,"Santa Clara, CA",t,1069,24807,2930,"{congressperson,Democrat}"
8,235312723,DesJarlaisTN04,Republican Congressman for Tennessee's 4th Con...,2011-01-07 21:47:44,NaN,t,1140,11975,3058,"{congressperson,Republican,republican}"
10,21406834,SenMarkey,Official account for Senator Edward J. Markey....,2009-02-20 15:49:29,"Malden, MA",t,802,53060,8681,"{senator,Democrat,democrat}"
13,21157904,SenatorBurr,"U.S. Senator from North Carolina, Chairman of ...",2009-02-18 01:53:03,"Winston-Salem, NC",t,1990,56998,3047,"{senator,Republican,republican}"


In [19]:
merged = pd.merge(pol_accounts, pol_tweets, left_on = "id", right_on = "user_id", how = "inner")
merged.head()

,id_x,screen_name,description,created_at_x,location,is_verified,latest_following_count,latest_followers_count,latest_status_count,array_agg,id_y,user_id,created_at_y,tweet_text,hashtag_entities,url_entities,favorites_count,retweet_count,quoted_status_id,in_reply_to_status_id
0,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",757957591171534848,2962813893,2016-07-26 15:15:46,Congrats to Chief Master Sgt Amy Giaquinto of ...,{},{https://t.co/vUJ7muCTy5},7,4,NaN,NaN
1,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",850019020891856902,2962813893,2017-04-06 16:15:01,ICYMI I'm working on legislative initiatives t...,{},{https://medium.com/@RepStefanik/supporting-mi...,5,2,NaN,NaN
2,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",758355089719496705,2962813893,2016-07-27 17:35:17,ICYMI: #NY21 Students register now to particip...,{NY21},{https://t.co/3CbW2bp1RV},1,2,NaN,NaN
3,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",666314203721019393,2962813893,2015-11-16 17:57:40,The @GOPpolicy hearing on The Essential Role o...,"{Millennials,GOPFuture}",{https://t.co/dWFSDnFJ1e},4,0,NaN,NaN
4,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",816312921219362816,2962813893,2017-01-03 15:59:01,Today Congress opens the 115th session. Lookin...,{NY21},{https://t.co/zT5Kj1tmCM},7,3,NaN,NaN


In [45]:
# test_suite = merged.iloc[0:9000]
party = []
for index, row in merged.iterrows(): # change from test_suite to merged
    array = row['array_agg']
    if "Republican" in array:
        party.append("Republican")
    elif "Democrat" in array:
        party.append("Democrat")
    else:
        party.append("None")
merged['party'] = party

In [47]:
merged = merged.join(pd.get_dummies(merged.party))

,id_x,screen_name,description,created_at_x,location,is_verified,latest_following_count,latest_followers_count,latest_status_count,array_agg,...,created_at_y,tweet_text,hashtag_entities,url_entities,favorites_count,retweet_count,quoted_status_id,in_reply_to_status_id,party,Republican
0,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",...,2016-07-26 15:15:46,Congrats to Chief Master Sgt Amy Giaquinto of ...,{},{https://t.co/vUJ7muCTy5},7,4,NaN,NaN,Republican,1
1,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",...,2017-04-06 16:15:01,ICYMI I'm working on legislative initiatives t...,{},{https://medium.com/@RepStefanik/supporting-mi...,5,2,NaN,NaN,Republican,1
2,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",...,2016-07-27 17:35:17,ICYMI: #NY21 Students register now to particip...,{NY21},{https://t.co/3CbW2bp1RV},1,2,NaN,NaN,Republican,1
3,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",...,2015-11-16 17:57:40,The @GOPpolicy hearing on The Essential Role o...,"{Millennials,GOPFuture}",{https://t.co/dWFSDnFJ1e},4,0,NaN,NaN,Republican,1
4,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",...,2017-01-03 15:59:01,Today Congress opens the 115th session. Lookin...,{NY21},{https://t.co/zT5Kj1tmCM},7,3,NaN,NaN,Republican,1


In [48]:
merged['party'].describe()
merged.party.value_counts()

count        1429738
unique             3
top       Republican
freq          728863
Name: party, dtype: object

In [49]:
merged.to_csv('merged.csv')